In [ ]:
import pyodbc
from Bio import SeqIO

# Function to parse a mixed FASTA file and extract protein sequences and relevant details
def parse_mixed_fasta(file_path):
    """
    Parses the mixed FASTA file to extract protein sequences and relevant details.

    Args:
        file_path (str): Path to the FASTA file.

    Returns:
        dict: A dictionary where keys are protein IDs and values are sequences.
        dict: A dictionary containing additional details extracted from the headers.
    """
    sequences = {}
    details = {}
    with open(file_path, 'r') as file:
        records = SeqIO.parse(file, 'fasta')
        for record in records:
            header = record.description  # Full header line
            sequence = str(record.seq)   # Sequence of the protein
            
            # Extract protein ID from the header
            protein_id = header.split('|')[1] if '|' in header else "Unknown"
            
            # Split header to extract protein name, organism type, and source
            parts = header.split(" ")
            protein_name = parts[1] if len(parts) > 1 else "Unknown"
            organism_type = parts[2] if len(parts) > 2 else "Unknown"
            
            # Determine the source based on the presence of specific terms in the header
            if "Homo sapiens" in header:
                source = "Homo sapiens"
            elif "Virus" in header:
                source = "Virus"
            else:
                source = "Unknown"
            
            # Store sequences and details
            sequences[protein_id] = sequence
            details[protein_id] = {
                "header": header,
                "protein_name": protein_name,
                "organism_type": organism_type,
                "source": source
            }
    return sequences, details

# Path to the mixed FASTA file
mixed_fasta_file = '/Users/user/Desktop/MIXED.fasta'

# Parse the mixed FASTA file
mixed_sequences, mixed_details = parse_mixed_fasta(mixed_fasta_file)

# Function to connect to SQL Server and create necessary tables
def connect_to_sql_server(server, database, Trusted_Connection):
    """
    Connects to SQL Server and creates necessary tables.

    Args:
        server (str): SQL Server name.
        database (str): Database name.
        Trusted_Connection (str): Trusted connection setting ('yes' or 'no').

    Returns:
        pyodbc.Connection: Connection object to the SQL Server.
    """
    connection_string = f'Driver={{ODBC Driver 17 for SQL Server}};SERVER={server};Trusted_Connection={Trusted_Connection}'
    conn = pyodbc.connect(connection_string)
    conn.autocommit = True  # Enable autocommit for database creation
    cursor = conn.cursor()

    # Create the database if it doesn't exist
    cursor.execute(f"IF NOT EXISTS (SELECT * FROM sys.databases WHERE name='{database}') CREATE DATABASE {database}")

    # Switch to the newly created or existing database
    cursor.execute(f"USE {database}")

    # Update the protein table schema to use VARCHAR for protein_id
    cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sys.tables WHERE name='protein')
    CREATE TABLE protein (
        protein_id VARCHAR(50) PRIMARY KEY,
        header VARCHAR(500),
        protein_name VARCHAR(500),
        organism_type VARCHAR(255),
        sequence TEXT,
        source VARCHAR(255)
    )
    """)

    # Ensure the protein_9_mer table uses VARCHAR for protein_id to match the protein table
    cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sys.tables WHERE name='protein_9_mer')
    CREATE TABLE protein_9_mer (
        mer CHAR(9),
        protein_id VARCHAR(50),
        position INT,
        FOREIGN KEY (protein_id) REFERENCES protein (protein_id)
    )
    """)

    return conn

# SQL Server connection details
server = 'PALOSKY'
database = 'ProteinDB'
Trusted_Connection = 'yes'

# Establish a connection to SQL Server and create the necessary tables
connection = connect_to_sql_server(server, database, Trusted_Connection)

# Function to insert protein data into the protein table
def insert_protein_data(connection, sequences, details):
    """
    Inserts protein data into the protein table.

    Args:
        connection (pyodbc.Connection): Connection object to the SQL Server.
        sequences (dict): Dictionary of protein sequences.
        details (dict): Dictionary of additional details for each sequence.
    """
    cursor = connection.cursor()
    for protein_id, sequence in sequences.items():
        header = details[protein_id]["header"]
        protein_name = details[protein_id]["protein_name"]
        organism_type = details[protein_id]["organism_type"]
        source = details[protein_id]["source"]
        cursor.execute(
            'INSERT INTO protein (protein_id, header, protein_name, organism_type, sequence, source) VALUES (?, ?, ?, ?, ?, ?)',
            protein_id, header, protein_name, organism_type, sequence, source
        )
    cursor.commit()

# Insert protein data into the protein table
insert_protein_data(connection, mixed_sequences, mixed_details)

# Function to generate all possible 9-mer records from each protein sequence
def generate_all_protein_9_mer_records(protein_sequences):
    """
    Generates all possible 9-mer records for protein sequences.

    Args:
        protein_sequences (dict): Dictionary of protein sequences.

    Returns:
        list: List of 9-mer records as tuples (mer, protein_id, position).
    """
    protein_9_mer_records = []
    for protein_id, sequence in protein_sequences.items():
        sequence_length = len(sequence)

        # Generate all possible consecutive 9-mers
        for i in range(sequence_length - 8):  # Ensure we do not go out of bounds
            mer = sequence[i:i + 9]
            position = i + 1  # 1-based indexing for positions
            protein_9_mer_records.append((mer, protein_id, position))

    return protein_9_mer_records

# Generate all possible 9-mer records
protein_9_mer_records = generate_all_protein_9_mer_records(mixed_sequences)

# Function to insert 9-mer data into the protein_9_mer table
def insert_protein_9_mer_data(connection, protein_9_mer_records):
    """
    Inserts 9-mer data into the protein_9_mer table.

    Args:
       connection (pyodbc.Connection): Connection object to the SQL Server.
        protein_9_mer_records (list): List of 9-mer records.
    """
    cursor = connection.cursor()
    for record in protein_9_mer_records:
        mer, protein_id, position = record
        cursor.execute('INSERT INTO protein_9_mer (mer, protein_id, position) VALUES (?, ?, ?)',
                       mer, protein_id, position)
    cursor.commit()

# Insert 9-mer data into the protein_9_mer table
insert_protein_9_mer_data(connection, protein_9_mer_records)

# Close the SQL Server connection
connection.close()
